### Data acquisition and preprocessing for pollutants 
 - lead, ozone, pm2.5

### imports

In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import requests as rq
from urllib.request import urlopen
import json
import os
import pathlib
import plotly.express as px
%matplotlib inline

In [2]:
# pandas formatting 
pd.set_option("display.max_rows", 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### functions

In [48]:
# functions

def FIPS_function(row):
    state = str(row['State Code']).zfill(2)
    county = str(row['County Code']).zfill(3)
    return str(state + county)
# convert to NO2 ug/m^3 for reference
def no2_mass_by_vol(ppb):
    ugm3 = 1.88*ppb
    return ugm3

def set_daily_cases_deaths(df):
    df['daily_new_cases'] = df['JHU_ConfirmedCases.data'].diff()
    df['daily_new_deaths'] = df['JHU_ConfirmedDeaths.data'].diff()
    return df

# plotting one day's avg 
def show_day_mean(df, date):
    fig = px.choropleth(df[df['Date Local']==date], geojson=counties, locations='fips', color='Arithmetic Mean',
                               color_continuous_scale="Plasma",
                               range_color=(0, 70), #max value for daily avg is ~60ppb
                               scope="usa",
                               labels={'Arithmetic Mean':'Arithmetic Mean (ppb)'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":1,'autoexpand':True })
    fig.update_layout(
        autosize=False,
        width=1200,
        height=900,
    )
    return fig

# plotting one day's max value
def show_day_max(df, date):
    fig = px.choropleth(df[df['Date Local']==date], geojson=counties, locations='fips', color='1st Max Value',
                               color_continuous_scale="Plasma",
                               range_color=(0, 70), #max value for daily avg is ~60ppb
                               scope="usa",
                               labels={'1st Max Value':'1st Max Value (ppb)'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":1,'autoexpand':True })
    fig.update_layout(
        autosize=False,
        width=1200,
        height=900,
    )
    return fig  

def show_sites(df):
    fig = px.choropleth(df, geojson=counties, locations='fips', color='Parameter Code',
                               color_continuous_scale="Plasma",
                               range_color=(0, 1), #max value for daily avg is ~60ppb
                               scope="usa",
                               labels={'1st Max Value':'1st Max Value (ppb)'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":1,'autoexpand':True })
    fig.update_layout(
        autosize=False,
        width=1200,
        height=900,
    )
    return fig

def get_counties_df(file_name='./data/covid/config/counties.json'):
    with open(file_name) as file:
                county_data = json.load(file)
    
    df = pd.DataFrame.from_dict(county_data)
    
    data = [df[col] for col in df.columns]    
    
    # pivot
    return pd.DataFrame(data,columns=df.index, index=df.columns)

def get_fips_from_county_lookup():
    counties = get_counties_df() 
    counties = counties.dropna(subset=['fips'])
    counties.fips = counties.fips.apply(lambda f: eval(str(f)).get('id'))
    counties_index = {k: v for k, v in zip(counties.index, counties.fips)}
    def get_fips_from_county(county):
        return counties_index[county]
    return get_fips_from_county

def avg_county_pollution(df, column_to_avg = "Arithmetic Mean", date_column=None):    
    if date_column == None:
        date_column = 'Date Local'
    # All days 
    # days = list(df[date_column].unique())
    # all_fips = list(df['fips'].unique())
    avg_by_day = df.groupby(['Date Local','fips']).agg({'Arithmetic Mean': 'mean','1st Max Value':'max','AQI':'max'})
    return avg_by_day

def cols_missing(df):
    total = 0
    for col in df.columns:
        missing_vals = df[col].isnull().sum()
        total += missing_vals
        pct = df[col].isna().mean() * 100
        if missing_vals != 0:
            print(f"{col} = {df[col].isnull().sum()}",'--','{} = {}%'.format(col, round(pct, 2)))
    if total == 0:
        print("no missing values left")
        
def print_outliers(df,col):
    print(col)
    #print(df[col].describe())
    print('Min: ',df[col].min())
    print('1st quartile: ',df[col].quantile(0.25))
    print('3rd quartile: ',df[col].quantile(0.75))
    print('Max: ',df[col].max())
    #print(sns.boxplot(x=df[col]))

### import data

In [59]:
#pm2.5 speciation
og_pm19 = pd.read_csv('data/daily_pm25_2019_FIPS.csv', dtype={'fips':'string'})
og_pm20 = pd.read_csv('data/daily_pm25_2020_FIPS.csv', dtype={'fips':'string'})
og_pm21 = pd.read_csv('data/daily_pm25_2021_FIPS.csv', dtype={'fips':'string'})

#lead
og_lead19 = pd.read_csv('data/daily_lead_2019_FIPS.csv', dtype={'fips':'string'})
og_lead20 = pd.read_csv('data/daily_lead_2020_FIPS.csv', dtype={'fips':'string'})
og_lead21 = pd.read_csv('data/daily_lead_2021_FIPS.csv', dtype={'fips':'string'})

#ozone
og_ozone19 = pd.read_csv('data/daily_ozone_2019_FIPS.csv', dtype={'fips':'string'})
og_ozone20 = pd.read_csv('data/daily_ozone_2020_FIPS.csv', dtype={'fips':'string'})
og_ozone21 = pd.read_csv('data/daily_ozone_2021_FIPS.csv', dtype={'fips':'string'})

### dropping columns

In [63]:
#dropping 
cols_to_drop = ['Pollutant Standard','POC','Parameter Code','Parameter Name','Sample Duration'
               , 'Method Code','Method Name','City Name','Date of Last Change']
#datum?

### pm2.5 quick look

In [64]:
pm19 = og_pm19.drop(cols_to_drop, axis=1)
pm20 = og_pm20.drop(cols_to_drop, axis=1)
pm21 = og_pm21.drop(cols_to_drop, axis=1)


print('2019 --->', pm19.shape,' 2020 ---> ',pm20.shape,' 2021 ---> ',pm21.shape,'\n')
print('Columns: ',pm19.columns)
pm19.head(1)

2019 ---> (540932, 21)  2020 --->  (554830, 21)  2021 --->  (55498, 21) 

Columns:  Index(['State Code', 'County Code', 'Site Num', 'Latitude', 'Longitude', 'Datum', 'Date Local', 'Units of Measure', 'Event Type', 'Observation Count', 'Observation Percent', 'Arithmetic Mean', '1st Max Value', '1st Max Hour', 'AQI', 'Local Site Name', 'Address', 'State Name', 'County Name', 'CBSA Name', 'fips'], dtype='object')


,State Code,County Code,Site Num,Latitude,Longitude,Datum,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Local Site Name,Address,State Name,County Name,CBSA Name,fips
0,1,3,10,30.497478,-87.880258,NAD83,2019-01-03,Micrograms/cubic meter (LC),None,1,100.0,4.3,4.3,0,18.0,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,"Daphne-Fairhope-Foley, AL",01003


### pm2.5 missing data with count and % of missing

In [65]:
#missing
print('--- pm2.5 2019 missing: ---')
print(cols_missing(pm19))
        
print(' \n --- pm2.5 2020 missing: ---')
print(cols_missing(pm20))

print('\n --- pm2.5 2021 missing: ---')
print(cols_missing(pm21))

--- pm2.5 2019 missing: ---
AQI = 230965 -- AQI = 42.7%
Local Site Name = 27587 -- Local Site Name = 5.1%
CBSA Name = 44318 -- CBSA Name = 8.19%
None
 
 --- pm2.5 2020 missing: ---
AQI = 243459 -- AQI = 43.88%
Local Site Name = 29791 -- Local Site Name = 5.37%
Address = 265 -- Address = 0.05%
CBSA Name = 43876 -- CBSA Name = 7.91%
None

 --- pm2.5 2021 missing: ---
AQI = 24524 -- AQI = 44.19%
Local Site Name = 3858 -- Local Site Name = 6.95%
CBSA Name = 5329 -- CBSA Name = 9.6%
None


In [37]:
# drop columns/rows if na 

#pm19['Arithmetic Mean'].dropna(inplace=True)
#pm20['Arithmetic Mean'].dropna(inplace=True)
#pm21['Arithmetic Mean'].dropna(inplace=True)

In [ ]:
#reduce memory usage

#change int64 to int32
#pm19['Arithmetic Mean'] = pm19['Arithmetic Mean'].astype('int32')

### pm25 outliers in artihmetic mean

In [66]:
#outliers
outlier_check = ['Arithmetic Mean']

for i in outlier_check:
    print('pm19: ');print(print_outliers(pm19,i))
    print('\n pm20: ');print(print_outliers(pm20,i))
    print('\n pm21: ');print(print_outliers(pm21,i))

pm19: 
Arithmetic Mean
Min:  -6.722222
1st quartile:  4.1375
3rd quartile:  9.4
Max:  327.0
None

 pm20: 
Arithmetic Mean
Min:  -5.0
1st quartile:  4.3
3rd quartile:  9.5
Max:  824.104167
None

 pm21: 
Arithmetic Mean
Min:  -3.3125
1st quartile:  4.416667
3rd quartile:  10.7125
Max:  222.420833
None


# Lead quick look

In [67]:
lead19 = og_lead19.drop(columns=cols_to_drop,axis=1)
lead20 = og_lead20.drop(columns=cols_to_drop,axis=1)
lead21 = og_lead21.drop(columns=cols_to_drop,axis=1)

print('2019 ---> ', lead19.shape,' 2020 ---> ',lead20.shape,' 2021 ---> ',lead21.shape,'\n')
print('Columns: ',lead19.columns)
lead19.head(1)

2019 --->  (12938, 21)  2020 --->  (10165, 21)  2021 --->  (442, 21) 

Columns:  Index(['State Code', 'County Code', 'Site Num', 'Latitude', 'Longitude', 'Datum', 'Date Local', 'Units of Measure', 'Event Type', 'Observation Count', 'Observation Percent', 'Arithmetic Mean', '1st Max Value', '1st Max Hour', 'AQI', 'Local Site Name', 'Address', 'State Name', 'County Name', 'CBSA Name', 'fips'], dtype='object')


,State Code,County Code,Site Num,Latitude,Longitude,Datum,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Local Site Name,Address,State Name,County Name,CBSA Name,fips
0,1,109,3,31.790479,-85.978974,NAD83,2019-01-03,Micrograms/cubic meter (LC),None,1,100.0,0.039,0.039,0,NaN,TROY LEAD,HENDERSON ROAD,Alabama,Pike,"Troy, AL",01109


### lead missing data with counts and %

In [68]:
print('--- lead 2019 missing: ---')
print(cols_missing(lead19))
        
print(' \n --- lead 2020 missing: ---')
print(cols_missing(lead20))

print('\n --- lead 2021 missing: ---')
print(cols_missing(lead21))

--- lead 2019 missing: ---
AQI = 12938 -- AQI = 100.0%
Local Site Name = 1698 -- Local Site Name = 13.12%
CBSA Name = 1018 -- CBSA Name = 7.87%
None
 
 --- lead 2020 missing: ---
AQI = 10165 -- AQI = 100.0%
Local Site Name = 1375 -- Local Site Name = 13.53%
CBSA Name = 1004 -- CBSA Name = 9.88%
None

 --- lead 2021 missing: ---
AQI = 442 -- AQI = 100.0%
Local Site Name = 29 -- Local Site Name = 6.56%
CBSA Name = 47 -- CBSA Name = 10.63%
None


In [ ]:
# drop columns/rows
# reduce memory?

### lead outliers in arithmetic mean

In [70]:
# #outliers
outlier_check = ['Arithmetic Mean']

for i in outlier_check:
    print('lead19: ');print(print_outliers(lead19,i))
    print('\n lead20: ');print(print_outliers(lead20,i))
    print('\n lead21: ');print(print_outliers(lead21,i))

lead19: 
Arithmetic Mean
Min:  0.0
1st quartile:  0.0026
3rd quartile:  0.018
Max:  1.991
None

 lead20: 
Arithmetic Mean
Min:  0.0
1st quartile:  0.003
3rd quartile:  0.019
Max:  1.202
None

 lead21: 
Arithmetic Mean
Min:  0.001
1st quartile:  0.002
3rd quartile:  0.01
Max:  0.228
None


# ozone quick look

In [71]:
ozone19 = og_ozone19.drop(columns=cols_to_drop,axis=1)
ozone20 = og_ozone20.drop(columns=cols_to_drop,axis=1)
ozone21 = og_ozone21.drop(columns=cols_to_drop,axis=1)

print('2019 ---> ',ozone19.shape,' 2020 ---> ',ozone20.shape,' 2021 ---> ',ozone21.shape,'\n')
print('Columns: ',ozone19.columns)
ozone19.head(1)

2019 --->  (392252, 21)  2020 --->  (391923, 21)  2021 --->  (37694, 21) 

Columns:  Index(['State Code', 'County Code', 'Site Num', 'Latitude', 'Longitude', 'Datum', 'Date Local', 'Units of Measure', 'Event Type', 'Observation Count', 'Observation Percent', 'Arithmetic Mean', '1st Max Value', '1st Max Hour', 'AQI', 'Local Site Name', 'Address', 'State Name', 'County Name', 'CBSA Name', 'fips'], dtype='object')


,State Code,County Code,Site Num,Latitude,Longitude,Datum,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Local Site Name,Address,State Name,County Name,CBSA Name,fips
0,1,3,10,30.497478,-87.880258,NAD83,2019-02-28,Parts per million,None,1,6.0,0.013,0.013,23,12,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,"Daphne-Fairhope-Foley, AL",01003


### ozone missing data with counts and %

In [72]:
print('--- ozone 2019 missing: ---')
print(cols_missing(ozone19))
        
print(' \n --- ozone 2020 missing: ---')
print(cols_missing(ozone20))

print('\n --- ozone 2021 missing: ---')
print(cols_missing(ozone21))

--- ozone 2019 missing: ---
Local Site Name = 19144 -- Local Site Name = 4.88%
CBSA Name = 41844 -- CBSA Name = 10.67%
None
 
 --- ozone 2020 missing: ---
Local Site Name = 19612 -- Local Site Name = 5.0%
Address = 132 -- Address = 0.03%
CBSA Name = 40854 -- CBSA Name = 10.42%
None

 --- ozone 2021 missing: ---
Local Site Name = 2842 -- Local Site Name = 7.54%
CBSA Name = 5209 -- CBSA Name = 13.82%
None


In [46]:
#drop columns/rows or reduce memory

### ozone arithmetic mean outlier

In [73]:
#outliers
outlier_check = ['Arithmetic Mean']

for i in outlier_check:
    print('ozone19: ');print(print_outliers(ozone19,i))
    print('\n ozone20: ');print(print_outliers(ozone20,i))
    print('\n ozone21: ');print(print_outliers(ozone21,i))

ozone19: 
Arithmetic Mean
Min:  -0.000667
1st quartile:  0.024647
3rd quartile:  0.039412
Max:  0.098235
None

 ozone20: 
Arithmetic Mean
Min:  -0.0033
1st quartile:  0.023235
3rd quartile:  0.038059
Max:  0.135529
None

 ozone21: 
Arithmetic Mean
Min:  0.0
1st quartile:  0.024706
3rd quartile:  0.037529
Max:  0.106583
None


### county mean

In [74]:
pm19_county_mean= pm19.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})
pm20_county_mean= pm20.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})
pm21_county_mean= pm21.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})

lead19_county_mean= lead19.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})
lead20_county_mean= lead20.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})
lead21_county_mean= lead21.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})

ozone19_county_mean= ozone19.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})
ozone20_county_mean= ozone20.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})
ozone21_county_mean= ozone21.groupby(['Date Local','fips']).agg({'Arithmetic Mean':'mean'})

### squash and reindex
    - drop 1st max for lead or would it be useful?

In [113]:
pm19_county_means = avg_county_pollution(pm19).drop(['1st Max Value'],axis=1).reset_index().groupby(['fips']).mean()
pm20_county_means = avg_county_pollution(pm20).drop(['1st Max Value'],axis=1).reset_index().groupby(['fips']).mean()
pm21_county_means = avg_county_pollution(pm21).drop(['1st Max Value'],axis=1).reset_index().groupby(['fips']).mean()

# drop 1st max?
lead19_county_means = avg_county_pollution(lead19).reset_index().groupby(['fips']).mean()
lead20_county_means = avg_county_pollution(lead20).reset_index().groupby(['fips']).mean()
lead21_county_means = avg_county_pollution(lead21).reset_index().groupby(['fips']).mean()

ozone19_county_means = avg_county_pollution(ozone19).drop(['1st Max Value'],axis=1).reset_index().groupby(['fips']).mean()
ozone20_county_means = avg_county_pollution(ozone20).drop(['1st Max Value'],axis=1).reset_index().groupby(['fips']).mean()
ozone21_county_means = avg_county_pollution(ozone21).drop(['1st Max Value'],axis=1).reset_index().groupby(['fips']).mean()

### convert to dictionary
    - can only do for pm and ozone, lead doesn't have an AQI

In [109]:
# pm25

pm19_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)
pm20_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)
pm21_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)

pm19_county_benchmarks = pm19_county_means.to_dict('index')
pm20_county_benchmarks = pm20_county_means.to_dict('index')
pm21_county_benchmarks = pm21_county_means.to_dict('index')

pm20_county_benchmarks['04013']['mean_ppb']

9.184634086235137

In [115]:
# lead

lead19_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)
lead20_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)
lead21_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)

lead19_county_benchmarks = lead19_county_means.to_dict('index')
lead20_county_benchmarks = lead20_county_means.to_dict('index')
lead21_county_benchmarks = lead21_county_means.to_dict('index')

lead20_county_benchmarks['04007']['mean_ppb']

0.011845410628019323

In [111]:
# ozone

ozone19_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)
ozone20_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)
ozone21_county_means.rename(columns={'Arithmetic Mean':'mean_ppb','AQI':'mean_AQI'},inplace=True)

ozone19_county_benchmarks = ozone19_county_means.to_dict('index')
ozone20_county_benchmarks = ozone20_county_means.to_dict('index')
ozone21_county_benchmarks = ozone21_county_means.to_dict('index')

ozone20_county_benchmarks['04013']['mean_ppb']

0.03571577303061661

In [99]:

def check_level(df, averages, delta=0):
    """Return dictonary with higher than average ppb for each fips code.
    
    
    Parameters:
    
    df -- single full dataframe
    averages -- dict of structure fips:(mean_ppb, AQI)
    delta -- ppb difference to flag, default = 0
    
    The averages dictionary should have one entry per fips code, 
            giving the average concentration for that county.
    
    Return:
    peaks -- dict of structure fips:[dates]
    """
    
    targets = {}
    for fipc in list(averages.keys()):
        df[df['fips']==fipc]
        peak_df = df[(df['fips']==fipc) & (df['Arithmetic Mean'] >= (averages[fipc]['mean_ppb'] + delta))]
        targets[fipc] = peak_df[['Date Local','Arithmetic Mean']].to_dict()
    return targets

### cross ref keys

In [119]:
#pm19

pm19_to_ref = check_level(pm19,pm19_county_benchmarks,delta=5)
pm20_to_ref = check_level(pm20,pm20_county_benchmarks,delta=5)
pm21_to_ref = check_level(pm21,pm21_county_benchmarks,delta=5)

pm19_cref_keys = list(pm19_to_ref.keys())
pm20_cref_keys = list(pm20_to_ref.keys())
pm21_cref_keys = list(pm21_to_ref.keys())

print('Number of pm19 keys: ',len(pm19_cref_keys))
print('Number of pm20 keys: ',len(pm20_cref_keys))
print('Number of pm21 keys: ',len(pm21_cref_keys))

Number of pm19 keys:  634
Number of pm20 keys:  626
Number of pm21 keys:  308


In [120]:
#lead 

lead19_to_ref = check_level(lead19,lead19_county_benchmarks,delta=5)
lead20_to_ref = check_level(lead20,lead20_county_benchmarks,delta=5)
lead21_to_ref = check_level(lead21,lead21_county_benchmarks,delta=5)

lead19_cref_keys = list(lead19_to_ref.keys())
lead20_cref_keys = list(lead20_to_ref.keys())
lead21_cref_keys = list(lead21_to_ref.keys())

print('Number of lead19 keys: ',len(lead19_cref_keys))
print('Number of lead20 keys: ',len(lead20_cref_keys))
print('Number of lead21 keys: ',len(lead21_cref_keys))

Number of lead19 keys:  80
Number of lead20 keys:  71
Number of lead21 keys:  19


In [121]:
#ozone
ozone19_to_ref = check_level(ozone19,ozone19_county_benchmarks,delta=5)
ozone20_to_ref = check_level(ozone20,ozone20_county_benchmarks,delta=5)
ozone21_to_ref = check_level(ozone21,ozone21_county_benchmarks,delta=5)

ozone19_cref_keys = list(ozone19_to_ref.keys())
ozone20_cref_keys = list(ozone20_to_ref.keys())
ozone21_cref_keys = list(ozone21_to_ref.keys())

print('Number of ozone19 keys: ',len(ozone19_cref_keys))
print('Number of ozone20 keys: ',len(ozone20_cref_keys))
print('Number of ozone21 keys: ',len(ozone21_cref_keys))

Number of ozone19 keys:  776
Number of ozone20 keys:  767
Number of ozone21 keys:  408


### Notes:
    - Not to much data for lead, plus aqi isn't recorded for it

### Next steps:
    - started a new notebook for it
    - going to find correlations between covid and pollutants above
    - attempt to find way to compare lead